In [9]:
!conda info

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

     active environment : bert
    active env location : /home/vzhekova/miniconda3/envs/bert
            shell level : 1
       user config file : /home/vzhekova/.condarc
 populated config files : /home/vzhekova/.condarc
          conda version : 23.7.2
    conda-build version : not installed
         python version : 3.9.16.final.0
       virtual packages : __archspec=1=x86_64
                          __cuda=11.2=0
                          __glibc=2.23=0
                          __linux=4.4.0=0
                          __unix=0=0
       base environment : /home/vzhekova/miniconda3  (writable)
      conda av data dir : /home/vzhekova/miniconda3/etc/conda
  conda 

In [24]:
PATH="/export/data4/vzhekova/biases-data/Test_De/Statistics/BERT"

In [6]:
# Test
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
filled = unmasker("Hello I'm a [MASK] model.")
for r in filled:
    print(r['token_str'], "->", r['score'])
print(filled[0]['token_str'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


fashion -> 0.10731080919504166
role -> 0.08774487674236298
new -> 0.05338389426469803
super -> 0.04667220637202263
fine -> 0.027095934376120567
fashion


# Extract sentences from MuST-SHE

In [25]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics/BERT


In [43]:
# Extract the 4th category of gender phenomena: 14M, 20F => 34 sentences total
!awk -F '\t' '$10 == "4F" || $10 == "4M" {print $5, $10}' en-fr.tsv > data.txt

# Masking with BERT

- For each sentence unmask each word

In [52]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')

# filled = unmasker("Hello I'm a [MASK] model.")
# print(filled[0]['token_str'])

with open('samples.txt') as in_file, open('test.txt', 'w') as out_file: 
    for line in in_file:
        sentence = line.split(' ')
        #print(sentence)
        for token in sentence:
            sentenceWithMask = list(map(lambda x: x.replace(token, '[MASK]'), sentence))
            #print(sentenceWithMask)
            unmasked = unmasker(' '.join(sentenceWithMask))
            #print(unmasked[0]['token_str'])
            newSentence = list(map(lambda x: x.replace('[MASK]', unmasked[0]['token_str']), sentenceWithMask))
            #print(newSentence)
            print(' '.join(newSentence), end='', file=out_file)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OSError: [Errno 28] No space left on device